Theory of likelihood analysis
===================

This jupyter notebook gives a basic overview of the theory of likelihood and maximum likelihood estimators applied to *Fermi* LAT. 

In the introduction to some of the probabilistic concepts below, I was blatantly inspired by the excellent [Bayesian methods in astronomy](https://github.com/jakevdp/BayesianAstronomy) tutorial. In fact, I am copying parts of Jake Vanderplas' tutorial below. You should go and read it if you have the chance.

Let's begin with some Python imports:

In [1]:
%pylab inline
import seaborn # for plot formatting

Populating the interactive namespace from numpy and matplotlib


# Fundamental questions of statistics

There are two fundamental types of statistical questions we want to answer:

**1. Model Fitting:** *Given this Model, what parameters best fit my data?*

Examples:

- What are the slope and intercept of a line of best-fit?
- What is the frequency, amplitude, and phase of a sinusoidal fit?

**2. Model Selection:** *Given two potential Models, which better describes my data?*

Examples:

- Does a linear or quadratic fit describe our data better?

Often one of the two models is a *null hypothesis*, or a baseline model in which the effect you're interested in is not observed.

# Likelihood definition

Here, we will focus on frequentist *maximum likelihood* approaches as a way of performing both model fitting and model selection. Another approach is Bayesian methods, but given our time constraints we will not cover them.

The starting point of maximum likelihood methods is to define the probability of seeing our data given the model—the likelihood:

$$ P(data ~|~ scientific\ model) $$ 

Let's define some symbols that will let us express this more easily:

$$
P(D ~|~ \theta)
$$

- $\theta$ represents the "science": the set of parameters that we are interested in constraining
- $D$ represents the "observed data"

It makes sense that the best-fit parameters that describe the data are those that maximize the likelihood defined above. Now all we need to do--as far as likelihood methods are concerned--is to compute the likelihood and maximize it. This should give us a point estimate of the model parameters that best describe the data.

# Simple example of statistical model

Since we want to maximize the likelihood, we need an expression to compute $P(D ~|~ \theta)$ for our data as a function of the parameters $\theta$.



If we were given:

- Data points $x_i, y_i$ with simple errorbars—this implies that probability for any *single* data point is a normal distribution about the true value
- Model $y_M(x; \theta)$ providing expected values

then

$$
y_i \sim \mathcal{N}(y_M(x_i;\theta), \sigma)
$$

and the likelihood would be

$$
P(x_i,y_i\mid\theta) = \frac{1}{\sqrt{2\pi\varepsilon_i^2}} \exp\left(\frac{-\left[y_i - y_M(x_i;\theta)\right]^2}{2\varepsilon_i^2}\right)
$$

where $\varepsilon_i$ are the (known) measurement errors indicated by the errorbars.

Assuming all the points are independent, we can find the *full likelihood by multiplying the individual likelihoods together*:

$$
P(D\mid\theta) = \prod_{i=1}^N P(x_i,y_i\mid\theta)
$$

which is a function of the model parameters and the data. From now on, we will refer to the likelihood function as $\mathcal{L}$:

- - - 
**Likelihood**
$$ \mathcal{L} \equiv  P(D\mid\theta). $$
- - - 

For convenience (and also for numerical accuracy) this is often expressed in terms of the *log-likelihood*, which for our simple example is:

$$
\log \mathcal{L} = \log P(D\mid\theta) = -\frac{1}{2}\sum_{i=1}^N \left \{ \log(2\pi\varepsilon_i^2) + \frac{\left[y_i - y_M(x_i;\theta)\right]^2}{\varepsilon_i^2} \right\}.
$$

### Exercises

1. Write a python method that creates some Mock data with errors bars, given a known line (i.e. known slope and intercept). We will fit this data below in order to recover the line parameters.

2. Write a Python function which computes the log-likelihood given a parameter vector $\theta$, an array of errors $\varepsilon$, and an array of $x$ and $y$ values

3. Use tools in [`scipy.optimize`](http://docs.scipy.org/doc/scipy/reference/optimize.html) to maximize this likelihood (i.e. minimize the negative log-likelihood). How close is this result to the input ``theta_true`` that you provided in exercise 1?

In [ ]:
# type answer here

## Useful references

- Bevington, Data reduction and analysis for the physical sciences
- Lyons, Statistics for nuclear and particle physicists

# Likelihood for *Fermi* LAT data

In our case, the input model is the distribution of gamma-ray sources on the sky and includes their intensity and spectra. We will bin the data in multidimensional bins—energy, sky pixels, time etc—because the counts are characterized by many variables. Thus, even with many counts, each bin will contain a small number of counts.

The observed number of counts in each bin $i$ is characterized by the [Poisson distribution](https://en.wikipedia.org/wiki/Poisson_distribution). With a small number of counts per bin, the Poisson distribution cannot be approximated by a normal distribution.

The likelihood $\mathcal{L}$ is the product of the probabilities of observing the detected counts in each bin. The detected counts in each bin is $n_i$, while the number of expected counts predicted by the model in the $i$th bin is $m_i$. Note that $m_i$ is a function of the source model, and will differ for different models. 

### Exercise 4

Assuming a Poisson distribution, write down the probability of detecting $n_i$ counts in a bin $i$, given the expected number of counts $m_i$. Try to work it out before proceeding with the text.

- - - 

In [ ]:
# type answer here

The probability of detecting $n_i$ counts in this bin is $P_i={m_i}^{n_i} e^{-m_i}/n_i!$. The likelihood $\mathcal{L}$ is the product of the probabilities $P_i$ of observing the detected counts in each bin for all $i$:

$$
\mathcal{L} = \prod_i P_i = \prod_i \frac{m_i^{n_i} e^{-m_i}}{n_i !}
$$

Using the properties of the product, $\mathcal{L}$ can be written in a slightly more convenient way:

- - -
**Binned likelihood**
$$\mathcal{L} = \prod_i e^{-m_i} \prod_i \frac{m_i^{n_i}}{n_i !} = e^{-N_{\rm pred}} \prod_i \frac{m_i^{n_i}}{n_i !} $$
- - - 

where $N_{\rm pred}$ is total number of counts that the source model predicts should have been detected.

The expression we just derived above is factored into $e^{-N_{\rm pred}}$ which is purely a function of the source model, and the product of $m_i^{n_i}/n_i!$ which is a function of both the source model and the data.

This likelihood, with finite size bins and $n_i$ that may be greater than 1, is the basis for **binned likelihood**. Since binning destroys information (i.e., the precise values of the quantities describing a count), there is a tradeoff between the number of bins (and thus the bin size) and the accuracy; smaller bins result in a more accurate likelihood.

If we let the bin sizes get infinitesimally small, then $n_i=0$ or 1. The likelihood is now 

- - - 
**Unbinned likelihood**
$$\mathcal{L} = e^{-N_{\rm pred}} \prod_i m_i $$
- - - 

Since $m_i$ is calculated using the precise values for each count—and not an average over a finite size bin—this **unbinned likelihood** is the most accurate.

For a small number of counts the unbinned likelihood can be calculated rapidly, but as the number of counts increases the time to calculate the likelihood becomes prohibitive, and the binned likelihood must be used.

Now what you have to do is to maximize $\mathcal{L}$ to get the best match of the model to the data. As [before](#simple-example-of-statistical-model), it is easier to handle and maximize the log-likelihood:

$$\log \mathcal{L} = \sum_i{\log m_i} -N_{\rm pred}  $$

## Explicit form of the likelihood 

Now we will open up a bit the likelihood function, in order to expose the complexities involved.

The source model is

$$S(E,\mathbf{p},t) = \sum_i s_i(E,t) \delta(\mathbf{p}-\mathbf{p_i}) +S_G(E,\mathbf{p})+S_{\rm eg}(E,\mathbf{p})+\sum_j S_j(E,\mathbf{p},t)$$

where each term has the following meaning:

![](./figures/source_model.png)

We need to fold the model with the [instrument response function (IRF)](https://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Cicerone/Cicerone_LAT_IRFs/IRF_overview.html) to obtain the predicted counts in the measured quantity space ($E',\mathbf{p}',t'$):

$$M(E',\mathbf{p}',t')= \int_{\rm SR} R(E',\mathbf{p}';E,\mathbf{p},t) S(E,\mathbf{p},t) \, dE \, d\mathbf{p}$$

where

$$R(E',\mathbf{p}';E,\mathbf{p},t) =  A(E,\mathbf{p}, \mathbf{L}(t)) D(E';E,\mathbf{p},\mathbf{L}(t)) P(\mathbf{p'};E,\mathbf{p},\mathbf{L}(t))$$

is the combined IRF (see below). $\mathbf{L}(t)$ is the orientation vector of the spacescraft. The integral is performed over the sky region (SR) encompassing all sources contributing to the Region-of–Interest (ROI). 

The function to maximize is

$$ \log \mathcal{L} = \sum_i \log M(E'_i,\mathbf{p_i'},t_i) - N_{\rm pred} $$

where the sum is performed over photons in the ROI. The predicted number of counts is 

$$ N_{\rm pred} = \int_{\rm SR} S(E,\mathbf{p}) \epsilon(E,\mathbf{p}) \, dE \, d\mathbf{p} $$.

### Saving CPU time

To save CPU time, a model-independent quantity—the exporuse map (cube)—is precomputed:

$$ \epsilon(E,\mathbf{p}) = \int_{\rm ROI} R(E',\mathbf{p'}; E,\mathbf{p},t) \, dE' \, d\mathbf{p'} \, dt $$

Finally, 

$$ N_{\rm pred} = \int_{\rm SR} S(E,\mathbf{p}) \epsilon (E,\mathbf{p}) \, dE \, d\mathbf{p'} $$


### Instrument response function

The IRF describes the telescope performance as a function of photon energy, incidence angle, conversion point within the instrument, and other important parameters. The IRF is in fact the mapping between the incoming photon flux and the detected events. 'Detection' depends not only on the LAT hardware but also on the processing that calculates the event parameters from the observables and assigns probabilities that an event is a photon. 

Indeed, the different event cuts are based on tradeoffs between the non-photon background, the effective area and the spatial and energy resolution; these cuts result in [analysis classes](https://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Cicerone/Cicerone_Data/LAT_DP.html).

# Significance of a gamma-ray source

We need to define an appropriate **test statistic (TS)** in order to quantify the statistical significance of emission of a source. We define the TS for Fermi LAT analysis as

- - - 
**Test statistic (TS)**
$$ TS = -2 \log \left( \frac{\mathcal{L}_{\rm max,0}}{\mathcal{L}_{\rm max,1}} \right) $$
- - - 

where $\mathcal{L}_{\rm max,0}$ is the maximum likelihood value for a model without an additional source (the *"null hypothesis"*) and $\mathcal{L}_{\rm max,1}$ is the maximum likelihood value for a model with the additional source at a specified location.

In the limit of a large number of counts, [Wilk’s Theorem](https://en.wikipedia.org/wiki/Likelihood-ratio_test#Wilks.27_theorem) states that the TS for the null hypothesis is asymptotically distributed as $\chi_n^2$ where $n$ is the number of parameters characterizing the additional source.
    
As a basic rule of thumb, the square root of the TS is approximately equal to the detection significance for a given source in standard deviations. 

### Example

$TS = 25$ is approximately equivalent to a statistical significance of emission from a source of $5 \sigma$. In other words, the probability that there is a source there is $99.9999\%$.

- - -

Now we will move on to the hands-on activity, where we will learn how to do a *binned likelihood fit in practice* with actual *Fermi* LAT data, in order to derive the best model parameters that describe the observations.

## Useful references

- [Likelihood overview](https://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Cicerone/Cicerone_Likelihood/Likelihood_overview.html), FSSC
- [General information about Fermi LAT analysis](https://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Cicerone/index.html)

[Solutions to exercises](fermi_likelihood_lecture-solutions.ipynb).